# Segmenting and Clustering Neighborhoods in Toronto

_this project is about exploring the neighberhood of Toranto. Firstly i will scrape the wikipedia page in order to get the needed data, Secondly i will analyze the data in order to affect to each psotal code the latitude and langutude information. Finnaly i will analyze the data using Kmean clustering and visualize the map of Toranto neigberhood._

### Import all needed library

In [15]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt

Libraries imported.


now we affect the link to the url variable 

In [19]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [20]:
page = urllib.request.urlopen(url)

In [21]:
soup = BeautifulSoup(page, "lxml")

get all tables in the web page 

In [22]:
 
all_tables=soup.find_all("table")


find the right tables in the web page 

In [23]:
right_table=soup.find('table', class_='wikitable sortable')


creat a for loop in order to extract the data from the web page and put in data frame tables named "df"

In [24]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))



In [25]:

df=pd.DataFrame(A,columns=['Postal code'])
df['Borough']=B
df['Neighborhood']=C

df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Analyse the data and eliminnate the messing values and display the six first rows.

In [26]:

df['Borough'].replace("Not assigned\n",'?', inplace = True)
df['Borough'].replace('?', np.nan, inplace=True)

df.dropna(subset=["Borough"], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

data_new=df.sort_values('Postal code')
data_new.head()


,Postal code,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


load the geocode of data using pandas lebrary 

In [27]:
adress="http://cocl.us/Geospatial_data"
data=pd.read_csv(adress)
data.sort_values('Postal Code')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


add the geogaphical code to data 

In [28]:
data_new['latitude']=data['Latitude']
data_new['longitude']=data['Longitude']
data_new.reset_index(drop=True, inplace=True)
data_new.head()

,Postal code,Borough,Neighborhood,latitude,longitude
0,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188


### Visualize the map of Toranto neighberhood 

In [12]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(data_new['latitude'],data_new['longitude'],data_new['Borough'],data_new['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

# cluster neighberhood

In [29]:
k=5
toronto_clustering = data_new.drop(['Postal code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
data_new.insert(0, 'Cluster Labels', kmeans.labels_)


### create a map of clusters

In [14]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(data_new['latitude'], data_new['longitude'], data_new['Neighborhood'], data_new['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters